- Create function to apply 

In [42]:
import os
import re
import shutil
import string
import datetime
import demoji
import numpy as np
import pandas as pd
#import spacy
from matplotlib import pyplot as plt
#from spacy.language import Language 
#from spacy_langdetect import LanguageDetector
#from langdetect import detect

In [3]:
# Listing all file paths in the Tweet Data folder 
data_path = os.listdir('Datasets/Tweet Data')
data_path.sort()
data_path = ['Datasets/Tweet Data/{}'.format(file) for file in data_path]
data_path = pd.DataFrame(data_path,columns=['File Path'])

# Getting the counter from the file name in to the dataframe
try:
    data_path = pd.concat([data_path,pd.DataFrame(data_path['File Path'].str.split(r'_',expand=True)[2])],axis=1)
    data_path = pd.concat([data_path,data_path[2].str.split(r'\.',expand=True)],axis=1,)
    data_path = data_path[['File Path',0]]
    data_path.columns = ['File Path', 'Counter']
    data_path['Counter'] = data_path['Counter'].apply(lambda x: int(x))
   
except:
    print('error')

In [59]:
# Reading the shape of the relevant excel sheet

from distutils.log import error
day = 0
total = 0
for i,file in enumerate(data_path.values):
    if np.mod(file[1],1440)==0:
        try:
            day +=1
            print(day,i,file[1],pd.read_csv(file[0],engine='python',index_col=0).shape)
            total += pd.read_csv(file[0],engine='python',index_col=0).shape[0]
            #shutil.copy(file[0],'Datasets/Daily Tweet Data/{}.csv'.format(str(datetime.datetime.now())))
        except error as e:
            print(e)

- Remove @[someone] - Done
- Remove string 'RT'- Done
- Remove new line - Done
- Remove punctuation (Remove # only; keep the string) - Done
- Remove links to http - Done
- Replace emojis by description - Done

In [10]:
def pre_process_tweet (text):
    #Remove all links starting with http...
    text = re.sub('https?:\/\/.*[\r\n]*','',text)
    #Remove RT
    text = re.sub('^RT[\s]+','',text)
    #Remove @[User]
    text = re.sub('@[^ ]+', '', text)
    #Remove Punctuation first
    text = text.translate(str.maketrans('','', string.punctuation))
    #Convert text to lower case
    text = text.lower()
    #Removing any number
    #text = re.sub('[0-9]+ ','',text)
    #Removing any number with text
    #text = re.sub('[0-9]+.+\s*','',text)
    text = re.sub(r'\w*\d\w*', '', text).strip()
    #Remove new line
    text = re.sub('\n',' ',text)
    #Replace emojis with description
    text = demoji.replace_with_desc(text,sep=' ')
    #Reducing whitespaces to one everywhere
    text = re.sub('\s+',' ',text)
    
    return text

In [45]:
# Listing all file paths in the Tweet Data folder 
data_path_daily = os.listdir('Datasets/Daily Tweet Data')
data_path_daily.sort()
data_path_daily = ['Datasets/Daily Tweet Data/{}'.format(file) for file in data_path_daily]
data_path_daily = pd.DataFrame(data_path_daily,columns=['File Path'])

# Getting the counter from the file name in to the dataframe
# try:
#     data_path = pd.concat([data_path,pd.DataFrame(data_path['File Path'].str.split(r'_',expand=True)[2])],axis=1)
#     data_path = pd.concat([data_path,data_path[2].str.split(r'\.',expand=True)],axis=1,)
#     data_path = data_path[['File Path',0]]
#     data_path.columns = ['File Path', 'Counter']
#     data_path['Counter'] = data_path['Counter'].apply(lambda x: int(x))
   
# except:
#     print('error')

In [51]:
clean_data = pd.DataFrame()
for i,file in enumerate(data_path_daily['File Path']):
    data = pd.read_csv(file,engine='python',index_col=0)
    data['tweed_id'] = data['tweed_id'].astype(str)
    data['clean_tweet'] = data['tweed_id'].apply(pre_process_tweet)
    clean_data = pd.concat([clean_data,data], ignore_index=True,axis=0)

clean_data.to_csv('clean_tweets.csv')


In [5]:
# def get_lang_detector(nlp, name):
#    return LanguageDetector()

# nlp = spacy.load("en_core_web_sm")
# Language.factory("language_detector", func = get_lang_detector)
# nlp.add_pipe('language_detector', last = True)

In [16]:
# def tweet_language(tweet):
#     document = nlp(tweet)
#     return document._.language['language']

In [7]:
# data['lang'] = data['clean_tweet'].apply(tweet_language)

In [8]:
# plt.bar(height = data['lang'].value_counts().values[:10],x=data['lang'].value_counts().index[:10])

In [9]:
# data['lang'].value_counts().index